<a href="https://colab.research.google.com/github/sumdher/SegmenterMovMedSpd/blob/main/Almost_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# READY with graph

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/ground_truth_09/tj_all.csv')
exhibits_df = pd.read_csv('/content/drive/MyDrive/ground_truth_09/POIs.csv')
df_gt = pd.read_csv('/content/drive/MyDrive/ground_truth_09/gt_stops.csv')

In [ ]:
# @title class MovMedSpeedSegmenter

import math
import pandas as pd
import numpy as np
import scipy.signal
from scipy.spatial.distance import cdist
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Point, MultiLineString
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HTML, VBox, Layout, HBox, Label, Button, Output, IntText
from IPython.display import display, clear_output
# import csv
# import time
# import seaborn as sns
# import pyproj


class MovMedSpeedSegmenter:

  def __init__(self, df, exhibits_df, df_gt = None):
    self.traj_df = df.copy()  # input trajectories dataframe
    self.exhibits_df = exhibits_df.copy()  # POIs location ana labels dataframe
    # self.groundtruth_df = df_gt.copy()  # (Optional) Groundtruth segments dataframe
    self.final_segments_df = None  # Output segments dataframe
    self.segments_df = None  # Output before clubbing and duration cut
    self.red_peaks_ = None
    self.blue_peaks_ = None
    self.red_peaks = None
    self.red_heights = None
    self.blue_peaks = None
    self.blue_heights = None

  def process_trajectory_data(self):
      """
      1. Adds features to traj_df: speed, movmedspd50, duration
      and distance between 2 consecutive points.
      2. Sorts, converts timestamps to datetime objects.
      """

      print("Total rows loaded: ", len(self.traj_df))
      self.exhibits_df['x'] = self.exhibits_df['x'] / 100
      self.exhibits_df['y'] = self.exhibits_df['y'] / 100
      self.exhibits_df = self.exhibits_df.sort_values(by=['name'])

      # self.groundtruth_df = self.groundtruth_df.sort_values(by=['person_id', 'start'])
      # self.groundtruth_df['start'] = pd.to_datetime(self.groundtruth_df['start'])
      # self.groundtruth_df['end'] = pd.to_datetime(self.groundtruth_df['end'])
      # self.groundtruth_df['dur'] = self.groundtruth_df.groupby('person_id').apply(
      #     lambda group: (group['end'] - group['start']).dt.total_seconds()
      #     ).reset_index(level=0, drop=True)

      self.traj_df = self.traj_df.sort_values(by=['person_id', 'time_string'])
      self.traj_df['time_string'] = pd.to_datetime(self.traj_df['time_string'])
      self.traj_df['n_point_id'] = self.traj_df.groupby('person_id').cumcount() + 1

      def _calculate_speed(group):
        group['duration'] = group[
            'time_string'
            ].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds()
        group['distance'] = (
            ((group['x_kf'].shift(-1) - group['x_kf']) ** 2 +
              (group['y_kf'].shift(-1) - group['y_kf']) ** 2) ** 0.5
        ).round(2)
        group['speed'] = (group['distance'] / group['duration']).round(2)
        group['speed'] = group['speed'].replace([float('inf'), -float('inf')], 0)
        return group
      self.traj_df = self.traj_df.groupby('person_id', group_keys=False).apply(_calculate_speed)

      self.traj_df['movmedspd_50'] = self.traj_df.groupby('person_id')['speed'].rolling(
          window=50, center=False
          ).median().reset_index(level=0, drop=True)

  def segment_graph(self, window_size=None):
      """
      1. Finds peaks and valleys in the moving median graph using Scipy's find_peaks():
         - high speed points (peaks): red_peaks_
         - low speed points (valleys): blue_peaks_ (by inverting the y-axis)
      2. Extracts segments: [peak, subsequent valley before next peak]
      3. Returns the segments_df
      """
      if window_size is None:
        window_size = 50

      self.traj_df['movmedspd'] = self.traj_df.groupby('person_id')['speed'].rolling(
          window=window_size, center=False
          ).median().reset_index(level=0, drop=True)

      self.red_peaks_ = self.traj_df.groupby('person_id').apply(
          lambda x: scipy.signal.find_peaks(
              x['movmedspd'], height=x['movmedspd'].median(), distance=20, width=20)
      )
      self.red_peaks = self.red_peaks_.apply(lambda x: x[0])
      self.red_heights = self.red_peaks_.apply(lambda x: x[1])
      self.blue_peaks_ = self.traj_df.groupby('person_id').apply(
          lambda x: scipy.signal.find_peaks(
              -x['movmedspd'], height=-np.median(self.red_heights[x.name]['peak_heights']),
              distance=20, width=20)
      )
      self.blue_peaks = self.blue_peaks_.apply(lambda x: x[0])
      self.blue_heights = self.blue_peaks_.apply(lambda x: x[1])

      segments = []
      for person_id, df_group in self.traj_df.groupby('person_id'):
          red_indices = self.red_peaks[person_id]
          blue_indices = self.blue_peaks[person_id]
          label = 1
          start_index = None
          end_index = None
          for i, row in enumerate(df_group.itertuples()):
              if row.n_point_id in red_indices and start_index is None:
                  # Red dot, start of a new segment
                  start_index = i
              elif row.n_point_id in blue_indices and start_index is not None:
                  # Blue cross after red, possible end of the current segment
                  end_index = i
              elif row.n_point_id in red_indices and start_index is not None:
                  if end_index is not None:
                      # Next segment red dot found
                      # Finalize the segment
                      segments.append(self._process_segment(
                          df_group, start_index, end_index, person_id, label)
                      )
                      label += 1
                      start_index = i
                      end_index = None
                  else:
                      # Red dot right after another red dot, no blue cross
                      # Restart segment from present red dot
                      start_index = i
                      end_index = None
          if start_index is not None and end_index is not None:
              segments.append(
                  self._process_segment(df_group, start_index, end_index, person_id, label)
              )
      self.segments_df = pd.DataFrame(segments, columns=[
          'person_id', 'label', 'x_centroid', 'y_centroid',
          'start_time', 'end_time', 'start_id', 'end_id', 'duration']
           )
      print("Number of raw segments before clubbing: ", len(self.segments_df))
      return self.segments_df

  def _segment_graph_dynamic(self, red_peak_params, blue_peak_params, window_size):
      """
      1. Finds segments by custom parameters
      2.  Used in scipy_peaks_plot(). Called when "Find Button" buttons clicked.
      """

      self.traj_df['movmedspd'] = self.traj_df.groupby('person_id')['speed'].rolling(
          window=window_size, center=False
          ).median().reset_index(level=0, drop=True)

      self.red_peaks_dyn = self.traj_df.groupby('person_id').apply(
          lambda x: scipy.signal.find_peaks(
              x['movmedspd'],
              height=red_peak_params['height'],
              distance=red_peak_params['distance'],
              width=red_peak_params['width'])[0]
      )

      self.blue_peaks_dyn = self.traj_df.groupby('person_id').apply(
          lambda x: scipy.signal.find_peaks(
              -x['movmedspd'],
              height=blue_peak_params['height'],
              distance=blue_peak_params['distance'],
              width=blue_peak_params['width'])[0]
      )

      segments = []
      for person_id, df_group in self.traj_df.groupby('person_id'):
          red_indices = self.red_peaks_dyn[person_id]
          blue_indices = self.blue_peaks_dyn[person_id]
          label = 1
          start_index = None
          end_index = None
          for i, row in enumerate(df_group.itertuples()):
              if row.n_point_id in red_indices and start_index is None:
                  # Red peak, start of a new segment
                  start_index = i
              elif row.n_point_id in blue_indices and start_index is not None:
                  # Blue peak after red, possible end of the current segment
                  end_index = i
              elif row.n_point_id in red_indices and start_index is not None:
                  if end_index is not None:
                      # Next segment red peak found
                      # Finalize the segment
                      segments.append(self._process_segment(
                          df_group, start_index, end_index, person_id, label)
                      )
                      label += 1
                      start_index = i
                      end_index = None
                  else:
                      # Red peak right after another red peak, no blue peak
                      # Restart segment from present red peak
                      start_index = i
                      end_index = None
          if start_index is not None and end_index is not None:
              segments.append(
                  self._process_segment(
                      df_group, start_index, end_index, person_id, label
                      )
              )
      self.segments_df = pd.DataFrame(segments, columns=[
          'person_id', 'label', 'x_centroid', 'y_centroid',
          'start_time', 'end_time', 'start_id', 'end_id', 'duration']
      )
      print("Number of raw segments before clubbing: ", len(self.segments_df))
      return self.segments_df

  def _process_segment(self, df_group, start_index, end_index, person_id, label):
      """Calculates centroids, start, end points, duration of segments."""

      start_time = df_group.iloc[start_index]['time_string']
      end_time = df_group.iloc[end_index]['time_string']
      start_id = df_group.iloc[start_index]['n_point_id']
      end_id = df_group.iloc[end_index]['n_point_id']
      segment_df = df_group.iloc[start_index:end_index + 1]
      x_centroid = segment_df['x_kf'].mean()
      y_centroid = segment_df['y_kf'].mean()
      duration = (end_time - start_time).total_seconds()
      return [person_id, label, x_centroid, y_centroid,
              start_time, end_time, start_id, end_id, duration
              ]

  def closest_exhibit_from_geom(self):
      """
      Finds closest exhibit to each segment's centriod.
      For exhibit type: geometric objects.
      Returns the segments_df with closest exhibit and the distance to it.
      """

      self.segments_df = self.segments_df.sort_values(by=['person_id', 'label'])
      def find_POI(row):
          point = Point(row['x_centroid'], row['y_centroid'])
          closest_exhibit = min(self.exhibits_df.itertuples(),
                            key=lambda exhibit: point.distance(exhibit.geometry))
          distance = point.distance(closest_exhibit.geometry)
          return pd.Series([closest_exhibit.exhibit_la, distance], index=['exhibit', 'exh_dist'])
      result_df = self.segments_df.apply(find_POI, axis=1)
      self.segments_df[['exhibit', 'exh_dist']] = result_df
      return self.segments_df

  def closest_exhibit_from_point(self):
      """
      Finds closest exhibit to each segment's centriod.
      For exhibit type: point coordinates.
      Returns the segments_df with closest exhibit and the distance to it.
      """

      self.segments_df = self.segments_df.sort_values(by=['person_id', 'label'])
      segments_array = self.segments_df[['x_centroid', 'y_centroid']].to_numpy()
      pois_array = self.exhibits_df[['x', 'y']].to_numpy()
      distances = cdist(segments_array, pois_array)
      closest_indices = distances.argmin(axis=1)
      self.segments_df['exhibit_o'] = 0
      self.segments_df['exhibit_o'] = self.exhibits_df.iloc[closest_indices]['name'].values
      self.segments_df['exh_dist'] = distances.min(axis=1)
      return self.segments_df

  def club_segments_df(self, cutoff_duration=None):
      """
      1. Clubs consecutive segments with same exhibit in proximity
      2. Optional: discards segments that have duration < cutoff_duration
      3. Recalculates centroids, updates start, end points and duration of segments.
      4. Returns the final_segments_df.
      """

      self.final_segments_df = (
          self.segments_df.merge(
              self.traj_df,
              left_on=['person_id', 'start_id'],
              right_on=['person_id', 'n_point_id']
          )
          .groupby([
              'person_id',
              'exhibit_o',
              (self.segments_df['exhibit_o'] != self.segments_df['exhibit_o'].shift()).cumsum()
          ])
          .agg(
              person_id=('person_id', 'first'),
              exhibit=('exhibit_o', 'first'),
              start_time=('start_time', 'first'),
              end_time=('end_time', 'last'),
              start_id=('start_id', 'first'),
              end_id=('end_id', 'last'),
              exh_dist=('exh_dist', 'mean')
          )
          .reset_index(level=[0, 1], drop=True)
          .reset_index()
      )
      def _calculate_centroids(row):
          points_in_segment = self.traj_df[
              (self.traj_df['person_id'] == row['person_id']) &
              (self.traj_df['n_point_id'] >= row['start_id']) &
              (self.traj_df['n_point_id'] <= row['end_id'])
          ]
          return pd.Series({
              'x_centroid': points_in_segment['x_kf'].mean(),
              'y_centroid': points_in_segment['y_kf'].mean()
          })

      self.final_segments_df[['x_centroid', 'y_centroid']] = self.final_segments_df.apply(
          _calculate_centroids, axis=1
          )

      self.final_segments_df = self.final_segments_df[
              (self.final_segments_df['exh_dist'] <= 1.5) |
              (self.final_segments_df['exhibit'] == 50)
              ]

      self.final_segments_df = self.final_segments_df.assign(
          label=lambda x: x.groupby('person_id').cumcount() + 1,
          start_time=lambda x: pd.to_datetime(x['start_time']),
          end_time=lambda x: pd.to_datetime(x['end_time']),
          duration=lambda x: (x['end_time'] - x['start_time']).dt.total_seconds()
      )

      if cutoff_duration is not None:
          self.final_segments_df = self.final_segments_df[
              self.final_segments_df['duration'] >= cutoff_duration
              ]
          print("Cut-off duration set: ", cutoff_duration)
      else:
          print("No cut_off duration set")
      print("Final segments after clubbing:", len(self.final_segments_df))


      self.traj_df['label'] = 0
      for _, segment1 in self.final_segments_df.iterrows():

        segment_range1 = (self.traj_df['person_id'] == segment1['person_id']) & \
                        (self.traj_df['n_point_id'] >= segment1['start_id']) & \
                          (self.traj_df['n_point_id'] <= segment1['end_id'])
        self.traj_df.loc[segment_range1, 'label'] = segment1['label']
      return self.final_segments_df

  def export_segments_csv(self, path=None):
    if self.final_segments_df is None:
      print("Finid the segments first!")
    if path is not None:
      self.final_segments_df.to_csv(path, index=True)
      print(f"Exported to {path}!")
    else:
      self.final_segments_df.to_csv('/content/final_segments.csv', index=True)
      print("Exported to /content/final_segments.csv!")

  def scipy_peaks_plot(self):
      """
      1. Choose parameters looking at the graph per eaach user.
      2. Overlay extracted segments on the graph
      3. Set duration cut-off and export to .csv file

      """
      segments_found = False
      median_seg = False
      unique_person_ids = self.traj_df['person_id'].unique().tolist()
      dropdown = widgets.Dropdown(
          options=unique_person_ids,
          value=unique_person_ids[0] if unique_person_ids else None,
          description='Person ID:',
      )
      dropdown.layout = Layout(margin='15px 0 15px 0')
      window_size_slider = widgets.IntSlider(value=50, min=1, max=200, step=5,
                                             description='Window Size:',
                                             layout=widgets.Layout(width='50%'))
      height_slider = widgets.FloatSlider(value=-0.8, min=-1.2, max=1.0, step=0.1,
                                          description='Height:',
                                          layout=widgets.Layout(width='40%'))
      prom_slider = widgets.FloatSlider(value=None, min=-2.0, max=2.0, step=0.01,
                                        description='Prominence:',
                                        layout=widgets.Layout(width='40%'))
      dist_slider = widgets.IntSlider(value=20, min=1, max=200, step=1,
                                      description='Distance:',
                                      layout=widgets.Layout(width='40%'))
      wid_slider = widgets.IntSlider(value=20, min=1, max=100, step=1,
                                     description='Width:',
                                     layout=widgets.Layout(width='40%'))
      height_slider1 = widgets.FloatSlider(value=0.3, min=-1.2, max=1.0, step=0.1,
                                           description='Height1:',
                                           layout=widgets.Layout(width='40%'))
      prom_slider1 = widgets.FloatSlider(value=None, min=-2.0, max=2.0, step=0.01,
                                         description='Prominence1:',
                                         layout=widgets.Layout(width='40%'))
      dist_slider1 = widgets.IntSlider(value=20, min=1, max=200, step=1,
                                       description='Distance1:',
                                       layout=widgets.Layout(width='40%'))
      wid_slider1 = widgets.IntSlider(value=20, min=1, max=100, step=1,
                                      description='Width1:',
                                      layout=widgets.Layout(width='40%'))

      btn_find_segments = Button(description='Find Segments')
      btn_export_csv = Button(description='Export to CSV')
      btn_find_segments_med = Button(description='Find Segments (medians)')
      # btn__update_plot = Button(description='Update Plot')

      plot_output = Output()
      output = Output()

      duration_text = IntText(
            value=8,
            description='Duration:',
            disabled=False,
            style={'description_width': 'initial'}
      )

      def _on_find_segments_click(_):
        nonlocal segments_found
        nonlocal median_seg
        with output:
            clear_output()
            print("Finding segments...")
            red_peak_params = {
                'height': height_slider1.value,
                'prominence': prom_slider1.value,
                'distance': dist_slider1.value,
                'width': wid_slider1.value
            }
            blue_peak_params = {
                'height': height_slider.value,
                'prominence': prom_slider.value,
                'distance': dist_slider.value,
                'width': wid_slider.value
            }
            segments_found = True
            duration = max(0, duration_text.value)
            window_size = window_size_slider.value
            self._segment_graph_dynamic(red_peak_params, blue_peak_params, window_size)
            self.closest_exhibit_from_point()
            self.club_segments_df(duration)
            median_seg = False
            _update_plot()

      def _on_export_csv_click(_):
        if self.final_segments_df is None:
          print("Find the segments first!")
        else:
          self.export_segments_csv()

      def _on_find_segments_median_click(_):
          nonlocal segments_found
          nonlocal median_seg
          with output:
              clear_output()
              print("Finding segments with Median-Speed + Peaks...")
              segments_found = True
              duration = max(0, duration_text.value)
              window_size = window_size_slider.value
              self.segment_graph(window_size)
              self.closest_exhibit_from_point()
              self.club_segments_df(duration)
              median_seg = True
              _update_plot()

      def _update_plot(change=None):
        nonlocal segments_found
        nonlocal median_seg
        with plot_output:
          plot_output.clear_output()
          # plot_output.clear_output(wait=True)
          selected_person_id = dropdown.value
          window_size = window_size_slider.value
          h = height_slider.value
          prom = prom_slider.value
          dist = dist_slider.value
          wid = wid_slider.value
          h1 = height_slider1.value
          p1 = prom_slider1.value
          d1 = dist_slider1.value
          w1 = wid_slider1.value

          selected_df = self.traj_df[self.traj_df['person_id'] == selected_person_id]
          x = selected_df['n_point_id']

          fig, ax = plt.subplots(figsize=(30, 5))

          if median_seg is False:
            y = selected_df['speed'].rolling(window=window_size).median()
            ax.plot(x, y, label='movMed_speed', color='green')
            peaks1 = scipy.signal.find_peaks(-y, height=h, distance=dist,
                                            prominence=prom, width=wid)
            peak_pos1 = peaks1[0]
            heights1 = peaks1[1]['peak_heights']

            peaks2 = scipy.signal.find_peaks(y, height=h1, distance=d1,
                                            prominence=p1, width=w1, plateau_size=None)
            peak_pos2 = peaks2[0]
            heights2 = peaks2[1]['peak_heights']
            ax.scatter(peak_pos1, -heights1, color='blue', s=30,
                      marker='x', label='Valleys')
            ax.scatter(peak_pos2, heights2, color='red', s=30,
                        marker='o', label='Peaks')
            if segments_found:
              mask = selected_df['label'] != 0
              y1 = np.where(mask, y, np.nan)
              ax.plot(x, y1, label='Detected segments', c='#ffbe0b')

          else:
            y = selected_df['speed'].rolling(window=50).median()
            ax.plot(x, y, label='movMed_speed', color='green')
            peak_pos1 = self.blue_peaks[selected_person_id]
            heights1 = self.blue_heights[selected_person_id]['peak_heights']
            peak_pos2 = self.red_peaks[selected_person_id]
            heights2 = self.red_heights[selected_person_id]['peak_heights']
            ax.scatter(peak_pos1, -heights1, color='blue', s=30,
                      marker='x', label='Valleys')
            ax.scatter(peak_pos2, heights2, color='red', s=30,
                        marker='o', label='Peaks')
            if segments_found:
              mask = selected_df['label'] != 0
              y1 = np.where(mask, y, np.nan)
              ax.plot(x, y1, label='Detected segments', c='#ffbe0b')

              if self.red_heights is not None and not self.red_heights.empty:
                blue_line_y = np.median(
                    self.red_heights[selected_df['person_id'].iloc[0]]['peak_heights']
                    )
                red_line_y = selected_df['movmedspd_50'].median()
              else:
                blue_line_y = 0
                red_line_y = 0

              ax.axhline(y=blue_line_y, color='blue', linewidth=1.3, label='Valleys threshold')
              ax.axhline(y=red_line_y, color='red', linewidth=1.3, label='Peaks threshold')



          ax.set_xlabel('point_id')
          ax.set_ylabel('MovMed_speed')
          ax.set_title(f'Moving median of speed. User: "{selected_person_id}"  \
                          Window Size: "{window_size}"')
          ax.legend()

          ax.grid(True, which='both', linestyle='--', linewidth=1)
          ax.set_xticks(np.arange(min(x), max(x)+1, 150))
          plt.show()


      duration_text.layout = Layout(margin='15px 0 15px 0')
      fndseg_layout = Layout(margin='15px 0 15px 0')
      btn_find_segments.layout = Layout(margin='15px 0 15px 0')
      btn_export_csv.layout = Layout(margin='5px 0 30px 0')
      btn_find_segments_med.layout = Layout(margin='5px 0 15px 0')
      btn_export_csv.on_click(_on_export_csv_click)
      btn_find_segments.on_click(_on_find_segments_click)
      btn_find_segments_med.on_click(_on_find_segments_median_click)
      # btn__update_plot.on_click(_update_plot)
      btn_find_segments.observe(_update_plot, names='value')  # type: ignore
      dropdown.observe(_update_plot, names='value') # type: ignore
      window_size_slider.observe(_update_plot, names='value') # type: ignore
      height_slider.observe(_update_plot, names='value') # type: ignore
      prom_slider.observe(_update_plot, names='value') # type: ignore
      dist_slider.observe(_update_plot, names='value') # type: ignore
      wid_slider.observe(_update_plot, names='value') # type: ignore
      height_slider1.observe(_update_plot, names='value') # type: ignore
      prom_slider1.observe(_update_plot, names='value') # type: ignore
      dist_slider1.observe(_update_plot, names='value') # type: ignore
      wid_slider1.observe(_update_plot, names='value') # type: ignore
      heading1 = HTML(
          value="<h3 style='font-size:20px; font-weight:bold; color:red;'>Adjust Peaks:</h3>"
          )
      heading2 = HTML(
          value="<h3 style='font-size:20px; font-weight:bold; color:#54c5e0;'>Adjust Valleys:</h3>"
          )

      blue_peaks_box = VBox([height_slider, prom_slider, dist_slider, wid_slider])
      red_peaks_box = VBox([height_slider1, prom_slider1, dist_slider1, wid_slider1])
      display(VBox([heading1, red_peaks_box, heading2, blue_peaks_box, dropdown,
                    window_size_slider, duration_text, btn_find_segments, btn_find_segments_med,
                    output, btn_export_csv, plot_output])
      )
      _update_plot(None)

In [ ]:
# @title Visualise Segments. Dynamic parameters with graph



object2 = MovMedSpeedSegmenter(df, exhibits_df)

object2.process_trajectory_data()
object2.scipy_peaks_plot()

In [ ]:
# @title Default fixed parameters. No graph.
import time

cutoff_duration = 8
window_length = 50

start_time = time.time()
object1 = MovMedSpeedSegmenter(df, exhibits_df)
object1.process_trajectory_data()
object1.segment_graph(window_length)  # Optional parameter: window_length. Default = 50
object1.closest_exhibit_from_point()
object1.club_segments_df(cutoff_duration)  # Optional parameter: cutoff_duration. Default = 0
# object1.export_segments_csv()

end_time = time.time()

total_time = end_time - start_time
print(f'time taken to run the algorithm: {total_time}')

Total rows loaded:  70090
Number of raw segments before clubbing:  284
Cut-off duration set:  8
Final segments after clubbing: 182
time taken to run the algorithm: 5.371860504150391


In [ ]:
# @title SPD class SegmenterSPD

import pandas as pd
import geopy.distance
from datetime import datetime, timedelta
import numpy as np
import math

class SegmenterSPD:

  def __init__(self, df):
    self.results_df = None
    self.traj_df = df.copy()

  def process_data(self):
    self.traj_df = self.traj_df.sort_values(by=['person_id', 'time_string'])
    self.traj_df['time_string'] = pd.to_datetime(self.traj_df['time_string'])
    self.traj_df['x_kf'] = self.traj_df['x_kf'].astype(float)
    self.traj_df['y_kf'] = self.traj_df['y_kf'].astype(float)
    self.traj_df.rename(columns={'x_kf': 'x', 'y_kf': 'y',
                                 'point_id': 'id', 'time_string': 'time_stamp'
                                 }, inplace=True)

  def find_seg_SPD(self, dist_limit, time_limit, dur=None):
    results = []
    for person_id, df_group in self.traj_df.groupby('person_id'):
        print(df_group.iloc[0]['person_id'])
        label = 1
        i = 0
        while i < len(df_group) - 1:
            for j in range(i + 1, len(df_group)):
                row_i = df_group.iloc[i]
                row_j = df_group.iloc[j]
                dist = math.sqrt((row_j['x'] - row_i['x'])**2 + (row_j['y'] - row_i['y'])**2)
                if dist > dist_limit:
                    tspan = abs((row_j['time_stamp'] - row_i['time_stamp']).total_seconds())
                    if tspan > time_limit:
                        lat_mean = df_group.iloc[i:j-1]['x'].mean()
                        lon_mean = df_group.iloc[i:j-1]['y'].mean()
                        new_row = {
                            'label': label,
                            'x_centroid': lat_mean,
                            'y_centroid': lon_mean,
                            'start_time': row_i['time_stamp'],
                            'end_time': row_j['time_stamp'],
                            'duration': tspan,
                            'person_id': row_i['person_id']
                        }
                        results.append(new_row)
                        label += 1
                    i = j
                    break
            i += 1
    self.results_df = pd.DataFrame(results)

    if dur:
      self.results_df = self.results_df[self.results_df['duration'] >= dur]
      self.results_df.reset_index(drop=True, inplace=True)
      self.results_df = self.results_df.assign(
          label=lambda x: x.groupby('person_id').cumcount() + 1
      )
      print("Total segments found: ", len(self.results_df))
      return self.results_df
    else:
      self.results_df.reset_index(drop=True, inplace=True)
      self.results_df = self.results_df.assign(
          label=lambda x: x.groupby('person_id').cumcount() + 1
      )
      print("Total segments found: ", len(self.results_df))
      return self.results_df


In [ ]:
import time

start_time = time.time()
objSPD = SegmenterSPD(df)

objSPD.process_data()

dist_limit = 1
time_limit = 1.5
dur = 8

stop_points_SPD = objSPD.find_seg_SPD(dist_limit, time_limit, dur)

end_time = time.time()

total_time = end_time - start_time

print(f'time taken to run: {total_time}')

T1_person1
T2_person1
T3_person11
T3_person13


In [ ]:
'########################'
'########################'
'###### Evaluation ######'
'########################'
'########################'

In [ ]:
%%capture
!pip install -q pulp

In [ ]:
# @title class MatchingRetriever1

from pulp import *
import os
import psycopg2
import numpy as np
import pandas as pd
import math

DURATION= 10
J_LIMIT = 0.0
ALFA=1
BETA=1-ALFA
JUST_FOR_PRINT=True
np.set_printoptions(threshold=sys.maxsize)


class MatchingRetriever1():

      def __init__(self, cl_df, gt_df):
        self.cl_df = cl_df.copy()
        self.gt_df = gt_df.copy()

      def process_prep(self):

        self.cl_df.sort_values(by=['person_id', 'label'], inplace=True)
        self.gt_df.sort_values(by=['person_id', 'gt_labeling'], inplace=True)

        self.cl_df['start_time'] = pd.to_datetime(self.cl_df['start_time'])
        self.cl_df['end_time'] = pd.to_datetime(self.cl_df['end_time'])
        self.gt_df['start'] = pd.to_datetime(self.gt_df['start'])
        self.gt_df['end'] = pd.to_datetime(self.gt_df['end'])

        self.cl_df['start_time_seconds'] = self.cl_df['start_time'].apply(lambda x: x.timestamp())
        self.cl_df['end_time_seconds'] = self.cl_df['end_time'].apply(lambda x: x.timestamp())

        self.gt_df['start_time_seconds'] = self.gt_df['start'].apply(lambda x: x.timestamp())
        self.gt_df['end_time_seconds'] = self.gt_df['end'].apply(lambda x: x.timestamp())

        gt_lists = []

        for person_id in self.gt_df['person_id'].unique():
            filtered_df = self.gt_df[self.gt_df['person_id'] == person_id]
            current_list = filtered_df.apply(
                lambda row: [row['gt_labeling'], row['start_time_seconds'], row['end_time_seconds'],
                             round((row['end_time_seconds'] - row['start_time_seconds']), 3),
                              (row['centroid_x_kf'], row['centroid_y_kf']), row['person_id']],
                axis=1
            ).tolist()
            gt_lists.append(current_list)

        cl_lists = []

        for person_id in self.cl_df['person_id'].unique():
            filtered_df = self.cl_df[self.cl_df['person_id'] == person_id]
            current_list = filtered_df.apply(
                lambda row: [row['label'], row['start_time_seconds'],row['end_time_seconds'],
                             round((row['end_time_seconds'] - row['start_time_seconds']), 3),
                              (row['x_centroid'], row['y_centroid']), row['person_id']],
                axis=1
            ).tolist()
            cl_lists.append(current_list)

        return gt_lists, cl_lists

      def find_match(self, gt_lists, cl_lists, results_df):
        results_list = []
        for gt, cl in zip(gt_lists, cl_lists):
          self.long_stops_count = 0
          self.short_stops_count = 0
          long_gt = []
          short_gt = []
          for g in gt:
            if g[3]>=DURATION:
              self.long_stops_count += 1
              long_gt.append(g)
            else:
              self.short_stops_count +=1
              short_gt.append(g)

          gt = long_gt

          if gt[0][-1] == cl[0][-1]:
            all_stops=[("S"+str(item[0]),item[1], item[2], item[3], item[4]) for item in gt]
            all_clusters=[(str(item[0]),item[1], item[2], item[3], item[4]) for item in cl]

            length=max(len(gt),len(cl))
            if len(gt)<len(cl):
              diff=len(cl)-len(gt)
              for i in range(diff):
                all_stops.append(("SNULL",-1,-1, -1, ("NULL", "NULL")))
            elif len(cl)<len(gt):
              diff=len(gt)-len(cl)
              for i in range(diff):
                all_clusters.append(("CNULL",-1,-1, -1, ("NULL", "NULL")))

            intersection_doublearray=[]

            stops_set=set()
            clusters_set=set()
            max_dist = 0.0

            for s in all_stops:
              l=[]
              for c in all_clusters:
                if (s[1]<=c[1] and c[1]<=s[2]) or (c[1]<=s[1] and s[1]<=c[2]):

                  intersection= min(c[2],s[2])-max(c[1],s[1])
                  union=max(c[2],s[2])-min(c[1],s[1])
                  j=float(intersection) / float(union)

                  if j * 100 > J_LIMIT: #SUD: J_LIMIT is zero, this condition now is always true, added once for experimental purposes
                    #SUD: You can ignore the dist here, it will make sense only if the evaluation is not only temporal
                    dist = math.dist(s[4], c[4])
                    if dist > max_dist - 0.001:
                      max_dist = dist + 0.001
                    if ALFA == 0:
                      l.append((j, dist))
                    elif ALFA == 1:
                      #l.append((int(j * 100), dist))
                      l.append((j, dist))
                    else:
                      l.append((j, dist))

                    stops_set.add(s[0])
                    clusters_set.add(c[0])

                  else:
                    l.append((0, 0))
                else:
                  l.append((0, 0))

              intersection_doublearray.append(l)

            l_alfa = []
            for row in intersection_doublearray:
              l_alfa_row = []
              for ts in row:
                if ts[0] > 0:
                  #l_alfa_row.append(ALFA * ts[0] + BETA * (1 - (ts[1] / max_dist)))
                  l_alfa_row.append(ALFA * ts[0] + BETA * (1 / (1+ts[1])))
                else:
                  l_alfa_row.append(0.0)
              l_alfa.append(l_alfa_row)

            # intersection_matrix=np.array(intersection_doublearray)
            intersection_matrix = np.array(l_alfa)

            # print(intersection_matrix)
            clusters = list(clusters_set)
            clusters.sort()
            stops = list(stops_set)
            stops.sort()

            #SUD: this is the part of the optimal assignement problem
            prob = LpProblem("Matching_stops", LpMaximize)
            y = LpVariable.dicts("pair", [(i,j) for i in range(length) for j in range(length)] ,cat='Binary')
            prob += lpSum([(intersection_matrix[i][j]) * y[(i,j)] for i in range(length) for j in range(length)])

            for i in range(length):
              prob += lpSum(y[(i,j)] for j in range(length)) <= 1
              prob += lpSum(y[(j,i)] for j in range(length)) <= 1
              prob += lpSum(y[(i,j)] + y[(j,i)] for j in range(length)) == 2
            prob += lpSum(y[(i,j)] for i in range(length) for j in range(length)) == length

            prob.solve()

            average_j = 0.0
            true_positive = 0
            false_negative = 0
            false_positive = 0
            true_negative=0
            intersection_short=0
            matched_avgs=[]
            sum_avg=0.0

            true_clusters = []
            true_detected_stops = []

            #SUD: here comes the work related to finding the matches
            #we use Y , the output of problem solving, and the intersection_matrix to make sure that the
            #stop and cluster matched are actually overlapping in time

            for i in range(length):
              for j in range(length):
                if y[(i, j)].varValue == 1:
                  if intersection_matrix[i, j] > 0:
                    if all_stops[i][3] >= DURATION:
                      average_j += intersection_matrix[i, j]
                      true_positive += 1
                      true_clusters.append(all_clusters[j])
                      true_detected_stops.append(all_stops[i])
                      # print('{} and {} with preference score {}'.format(all_stops[i], all_clusters[j],
                                                # intersection_matrix[i, j]))
                      intersection = min(all_clusters[j][2], all_stops[i][2]) - max(all_clusters[j][1],
                                                      all_stops[i][1])
                      union = max(all_clusters[j][2], all_stops[i][2]) - min(all_clusters[j][1], all_stops[i][1])
                      jacc = float(intersection) / float(union)
                      matched_avgs.append((all_stops[i], all_clusters[j], jacc))
                      sum_avg += jacc

                    elif all_stops[i][3] > 0:
                      intersection_short += 1




            true_negative = self.short_stops_count - intersection_short
            false_positive = len(cl)-true_positive
            false_negative = self.long_stops_count - true_positive

            #SUD: now we obtained TP, FP and FN, enough for calculating F-score
            #In the following we are detailing and classifying the FP and FN,
            #You may ignore it for the moment


            #########fp###############
            false_clusters = [x for x in all_clusters if x not in true_clusters]
            fp_split = 0
            fp_short = 0
            fp_split_label=[]
            fp_short_label=[]
            for c in false_clusters:
              is_split=False
              for s in long_gt:
                if (s[1] <= c[1] and c[1] <= s[2]) or (c[1] <= s[1] and s[1] <= c[2]):  # there is an intersection
                  fp_split += 1
                  fp_split_label.append(c[0])
                  is_split=True
                  break
              if not is_split:
                for s in short_gt:
                  if (s[1] <= c[1] and c[1] <= s[2]) or (c[1] <= s[1] and s[1] <= c[2]):  # there is an intersection
                    fp_short += 1
                    fp_short_label.append(c[0])
                    break

            fp_move = false_positive - fp_split - fp_short

            #########fn##############
            false_stops = [x for x in all_stops if x not in true_detected_stops]
            fn_merge = 0
            fn_merge_labels = []
            fn_fail = 0

            for s in false_stops:
              for c in all_clusters:
                if (s[1] <= c[1] and c[1] <= s[2]) or (c[1] <= s[1] and s[1] <= c[2]):  # there is an intersection
                  fn_merge += 1
                  fn_merge_labels.append(s[0])
                  break

            fn_fail = false_negative - fn_merge

            ############metrics#############

            #SUD: here we compute the Fscore an the Sscore(avg_jacc)
            if true_positive > 0:
              avg_jacc=sum_avg/len(matched_avgs)
              average_j = average_j / (100 * true_positive)

              recall = true_positive / (true_positive + false_negative)
              precision = true_positive / (true_positive + false_positive)
              f_measure = 2 * recall * precision / (recall + precision)
              # print(f_measure)

            else:
              avg_jacc = 0
              average_j = 0
              recall = 0
              precision = 0
              f_measure = 0
            # print(f_measure)

            accuracy = float(true_positive + true_negative) / float(true_positive + true_negative + false_positive + false_negative)

            new_row = {
                "accuracy": accuracy,
                "true_positive": true_positive,
                "true_negative": true_negative,
                "false_positive": false_positive,
                "false_negative": false_negative,
                "f_measure": f_measure,
                "recall": recall,
                "precision": precision,
                "avg_jacc": avg_jacc,
                "person_id": gt[0][-1]
            }

            results_list.append(new_row)
            # results_df = results_df.append(new_row, ignore_index=True)

            # print("short stops ", short_gt)
            # print("tp, tn, fp, fn, fmeasure, avg_jacc")
            # print(true_positive, true_negative, false_positive, false_negative, f_measure, avg_jacc)
            # print("FP because of = splitting, short stops, move")
            # print(false_positive, "=", fp_split, fp_short, fp_move)
            # print("FP_short =", fp_short_label)
            # print("FP_split =", fp_split_label)
        results_df = pd.DataFrame(results_list)
        return results_df


In [ ]:
### SPD

cl_df = stop_points_SPD

m = MatchingRetriever1(cl_df, df_gt)

gt_lists, cl_lists = m.process_prep()

results_df_SPD = pd.DataFrame()
results_df_SPD = m.find_match(gt_lists, cl_lists, results_df_SPD)

In [ ]:
### MOvMedSpeed

cl_df = object1.final_segments_df

m = MatchingRetriever1(cl_df, df_gt)

gt_lists, cl_lists = m.process_prep()

results_df_MMS = pd.DataFrame()
results_df_MMS = m.find_match(gt_lists, cl_lists, results_df_MMS)


In [ ]:
results_df_MMS

,accuracy,true_positive,true_negative,false_positive,false_negative,f_measure,recall,precision,avg_jacc,person_id
0,0.903226,26,2,2,1,0.945455,0.962963,0.928571,0.726347,T1_person1
1,0.785714,22,0,0,6,0.880000,0.785714,1.000000,0.778257,T2_person1
2,0.692308,17,1,1,7,0.809524,0.708333,0.944444,0.635063,T3_person11
3,0.913043,17,4,2,0,0.944444,1.000000,0.894737,0.568140,T3_person13
4,0.950000,19,0,1,0,0.974359,1.000000,0.950000,0.585458,T3_person15
5,0.692308,18,0,0,8,0.818182,0.692308,1.000000,0.722243,T4_person11
6,0.896552,23,3,1,2,0.938776,0.920000,0.958333,0.716871,T4_person13
7,1.000000,22,0,0,0,1.000000,1.000000,1.000000,0.786805,T4_person15
8,0.833333,10,0,1,1,0.909091,0.909091,0.909091,0.722791,T_old_v


In [ ]:
print(results_df_MMS.f_measure.mean(), '+-', results_df_MMS.f_measure.std())

0.913314445695398 +- 0.06599389363171686


In [ ]:
results_df_MMS[['accuracy', 'f_measure',	'recall',	'precision',	'avg_jacc']].mean()

accuracy     0.851832
f_measure    0.913314
recall       0.886490
precision    0.953909
avg_jacc     0.693553
dtype: float64

In [ ]:
results_df_SPD[['accuracy', 'f_measure',	'recall',	'precision',	'avg_jacc']].mean()

accuracy     0.812062
f_measure    0.890523
recall       0.880263
precision    0.910722
avg_jacc     0.724472
dtype: float64

In [ ]:
results_df_SPD.f_measure.mean()

0.8905232295406805

In [ ]:
results_df_SPD

,accuracy,true_positive,true_negative,false_positive,false_negative,f_measure,recall,precision,avg_jacc,person_id
0,0.862069,23,2,0,4,0.920000,0.851852,1.000000,0.662645,T1_person1
1,0.821429,23,0,0,5,0.901961,0.821429,1.000000,0.703697,T2_person1
2,0.884615,22,1,1,2,0.936170,0.916667,0.956522,0.741428,T3_person11
3,0.769231,16,4,5,1,0.842105,0.941176,0.761905,0.723472,T3_person13
4,0.809524,17,0,2,2,0.894737,0.894737,0.894737,0.736763,T3_person15
5,0.769231,20,0,0,6,0.869565,0.769231,1.000000,0.683398,T4_person11
6,0.903226,25,3,3,0,0.943396,1.000000,0.892857,0.764776,T4_person13
7,0.720000,18,0,3,4,0.837209,0.818182,0.857143,0.735678,T4_person15
8,0.769231,10,0,2,1,0.869565,0.909091,0.833333,0.768395,T_old_v


In [ ]:
print(results_df_SPD.avg_jacc.mean(), '+-', results_df_SPD.avg_jacc.std())

0.7244723779390954 +- 0.03546142225146571


In [ ]:
!pip install -q ipdb